In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import curve_fit
import warnings
from scipy.optimize import differential_evolution

i = 0
N = 25 #number of measurements
Max = 1700 #maximum index of fitted Raman peak
Min = 1500 #minimum index of fitted Raman peak
name = 'mapa_'
output = 'output_file_'

def double_Lorentz(x, a, b, A, w, x_0, A1, w1, x_01):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))+(2*A1/np.pi)*(w1/(4*(x-x_01)**2 + w1**2))

def Lorentz(x, a, b, A, w, x_0):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))


def sumOfSquaredError(parameterTuple):
    warnings.filterwarnings("ignore") # do not print warnings by genetic algorithm
    return np.sum((intens_plot - double_Lorentz(wave_plot, *parameterTuple)) ** 2)

def generate_Initial_Parameters():
    # min and max used for bounds
    maxX = max(wave_plot)
    minX = min(wave_plot)
    maxY = max(intens_plot)
    minY = min(intens_plot)
    
    parameterBounds = []
    parameterBounds.append([-1.0, 1.0]) # parameter bounds for a
    parameterBounds.append([maxY/-2.0, maxY/2.0]) # parameter bounds for b
    parameterBounds.append([0.0, maxY*100.0]) # parameter bounds for A
    parameterBounds.append([0.0, maxY/2.0]) # parameter bounds for w
    parameterBounds.append([1550, 1570]) # parameter bounds for x_0
    parameterBounds.append([0.0, maxY*100.0]) # parameter bounds for A1
    parameterBounds.append([0.0, maxY/2.0]) # parameter bounds for w1
    parameterBounds.append([1575, 1599]) # parameter bounds for x_01

    # "seed" the numpy random number generator for repeatable results
    result = differential_evolution(sumOfSquaredError, parameterBounds, seed=3)
    return result.x


for m in range(0,N):

    data = np.loadtxt(name + str(m) +'.txt')
    x = data[:,0]
    y = data[:,1]
    wave = data[:,2]
    intensity = data[:,3]

    list_waves, list_intensities, list_x, list_y = [], [], [], []
    current_session_beggining = 0
    wave_diff_bool = (wave[1:] - wave[:-1]) > 0

    beggining_indices = np.where(wave_diff_bool)
    beggining_indices = np.append(0, beggining_indices) #zero_index
    beggining_indices = np.append(beggining_indices, len(wave)) #first_index
    
    for i in range(len(beggining_indices[:-1])):
        beginning = beggining_indices[i]
        ending = beggining_indices[i+1]
        if i==0:
            current_wave = wave[beginning : ending+1]
            current_intensity = intensity[beginning : ending+1]
            current_x = x[beginning : ending+1]
            current_y = y[beginning : ending+1]
        else:
            current_wave = wave[beginning+1 : ending+1]
            current_intensity = intensity[beginning+1 : ending+1]
            current_x = x[beginning+1 : ending+1]
            current_y = y[beginning+1 : ending+1]
        
        list_waves.append(current_wave)
        list_intensities.append(current_intensity)
        list_x.append(current_x)
        list_y.append(current_y)
        

    n = 0
    index_G_end=0
    index_G_begin=0
    for i in range(len(list_waves)): #here we choose which of the Raman peaks will be fitted
        while list_waves[i][n] > Max: #this examle was made for carbon nanotubes G mode
            n = n+1
            index_G_begin=n
        while list_waves[i][n] > Min:
            n = n+1
            index_G_end=n
        n=1
    
    for i in range(len(list_waves)):
        intens_plot =list_intensities[i][index_G_begin:index_G_end]
        wave_plot =list_waves[i][index_G_begin:index_G_end] 

    # generate initial parameter values
    initialParameters = generate_Initial_Parameters()
    x_zeros, w_G, A_G = [], [], []

    for i in range(len(list_waves)): #fitting double Lorentz function to G mode
        intens_plot =list_intensities[i][index_G_begin:index_G_end]
        wave_plot =list_waves[i][index_G_begin:index_G_end]

        parametry, niepewnosci = curve_fit(double_Lorentz, wave_plot, intens_plot, initialParameters)
        a, b, A, w, x_0, A1, w1, x_01 = parametry
        x_fit = np.linspace(wave_plot.min(), wave_plot.max(), 1000)
        y_fit = double_Lorentz(x_fit, a, b, A, w, x_0, A1, w1, x_01)
        x_zeros.append(x_01)
        w_G.append(w1)
        A_G.append(A1)
    
    plt.plot(x_fit, y_fit, wave_plot, intens_plot) #allows to see how well the curve has been fitted however will open a plot in separate window for each measurement
    plt.show() 

    powdata=open(output+str(m)+".txt","w")
    for val in zip(x_zeros, w_G, A_G):
        powdata.write('{} \t {} \t {}\n'.format(val[0], val[1], val[2]))
    powdata.close()